In [1]:
import os
os.chdir("F:\Deepclassifier_project")

In [2]:
CONFIG_FILE_PATH

NameError: name 'CONFIG_FILE_PATH' is not defined

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    
@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

    

In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml , create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
                                    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self)-> TrainingConfig:
        training = self.config.training
        params = self.params
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path=Path(self.config.prepare_base_model.updated_base_model_path),
            training_data= os.path.join(self.config.data_ingestion.unzip_dir,"PetImages"),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
                             )
                            
        create_directories([training.root_dir])

        return training_config
        
    def get_prepare_callback_config(self)-> PrepareCallBackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])
        params = self.params
        prepare_callbacks_config = PrepareCallBackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
                             )
        create_directories([config.root_dir])

        return prepare_callbacks_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


class PrepareCallback:
    def __init__(self,config:PrepareCallBackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):


        return tf.keras.callbacks.ModelCheckpoint(
        filepath=self.config.checkpoint_model_filepath,
        save_best_only = True)

    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_ckpt_callbacks,
            self._create_ckpt_callbacks
        ]


class Training:
    def __init__(self,config:TrainingConfig,epochs):
        self.config = config
        self.epochs = epochs

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split =0.20
                )
           
        dataflow__kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation= "bilinear"
                                )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs,)

        self.valid_generator = valid_datagenerator.flow_from_directory(directory=self.config.training_data,
                                subset="validation",
                                shuffle=False,
                                **dataflow__kwargs
                                )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
                                )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(directory=self.config.training_data,
                            subset="training",
                            shuffle=True,
                            **dataflow__kwargs
                                )
    @staticmethod
    def save_model(path:Path,model:tf.keras.models):
        model.save(path)

    def train(self,callbacks_list:list):
        self.step_per_epoch = self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(self.train_generator,epochs=self.epochs,steps_per_epoch=self.step_per_epoch,validation_steps=self.validation_steps,
        validation_data = self.valid_generator,callbacks=callbacks_list)

        self.save_model(path=self.config.trained_model_path,model=self.model)


In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()   
    
    training_config= config.get_training_config()
    training = Training(config=training_config,epochs=config.params.EPOCHS)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callbacks_list=callback_list) 
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 604/1250 [=============>................] - ETA: 24:17 - loss: 8.4218 - accuracy: 0.6270

f:\Deepclassifier_project\env\lib\site-packages\PIL\TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 3408s 3s/step - loss: 6.3072 - accuracy: 0.6855 - val_loss: 3.4843 - val_accuracy: 0.7794


AttributeError: 'TrainingConfig' object has no attribute 'params'

In [24]:
os.getcwd()

'F:\\Deepclassifier_project\\src\\deepClassifier'